# Dude, Where's My House?
## Part 4: Training Models
## Authors: Aman Hafez, Kavan Pandya, Yan Nusinovich
Notebook drafted by Yan Nusinovich

## References

- https://www.analyticsvidhya.com/blog/2020/02/learn-image-classification-cnn-convolutional-neural-networks-3-datasets/
- https://medium.com/@vijayabhaskar96/tutorial-image-classification-with-keras-flow-from-directory-and-generators-95f75ebe5720
- https://machinelearningmastery.com/how-to-load-large-datasets-from-directories-for-deep-learning-with-keras/

## Code

In [41]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPool2D, Flatten, Dense, InputLayer, BatchNormalization, Dropout
from keras.preprocessing.image import ImageDataGenerator
from keras.applications import VGG16
from keras.utils import to_categorical
from keras.callbacks import EarlyStopping
import numpy as np
import pandas as pd
import pickle

## Model 1

In [2]:
imagegen = ImageDataGenerator()
train = imagegen.flow_from_directory("data/train/", class_mode="categorical", shuffle=True, batch_size=64, target_size=(256, 256))
val = imagegen.flow_from_directory("data/validation/", class_mode="categorical", shuffle=True, batch_size=64, target_size=(256, 256))


Found 820 images belonging to 3 classes.
Found 423 images belonging to 3 classes.


In [11]:
model = Sequential()
model.add(InputLayer(input_shape=(256, 256, 3)))

model.add(Conv2D(25, (5, 5), activation='relu', strides=(1, 1), padding='same')) # add batch size and ES
model.add(MaxPool2D(pool_size=(2, 2), padding='same'))
model.add(Conv2D(50, (5, 5), activation='relu', strides=(2, 2), padding='same'))
model.add(MaxPool2D(pool_size=(2, 2), padding='same'))
model.add(BatchNormalization())
model.add(Conv2D(70, (3, 3), activation='relu', strides=(2, 2), padding='same'))
model.add(MaxPool2D(pool_size=(2, 2), padding='valid'))
model.add(BatchNormalization())
model.add(Flatten())
model.add(Dense(units=100, activation='relu'))
model.add(Dense(units=100, activation='relu'))
model.add(Dropout(0.25))
model.add(Dense(units=3, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer="adam", metrics=['accuracy'])
model.fit_generator(train, epochs=30, validation_data=val)

Epoch 1/30
13/13 [==============================] - 118s 9s/step - loss: 1.1750 - accuracy: 0.4646 - val_loss: 21.5300 - val_accuracy: 0.3712
Epoch 2/30
13/13 [==============================] - 115s 9s/step - loss: 0.7885 - accuracy: 0.6427 - val_loss: 5.9983 - val_accuracy: 0.3759
Epoch 3/30
13/13 [==============================] - 116s 9s/step - loss: 0.5304 - accuracy: 0.7890 - val_loss: 7.6906 - val_accuracy: 0.3783
Epoch 4/30
13/13 [==============================] - 114s 9s/step - loss: 0.3754 - accuracy: 0.8610 - val_loss: 4.3909 - val_accuracy: 0.4113
Epoch 5/30
13/13 [==============================] - 116s 9s/step - loss: 0.2088 - accuracy: 0.9402 - val_loss: 5.9223 - val_accuracy: 0.4374
Epoch 6/30
13/13 [==============================] - 115s 9s/step - loss: 0.1571 - accuracy: 0.9500 - val_loss: 4.5432 - val_accuracy: 0.4279
Epoch 7/30
13/13 [==============================] - 117s 9s/step - loss: 0.1036 - accuracy: 0.9744 - val_loss: 0.8198 - val_accuracy: 0.5863
Epoch 8/30
1

## Model 2

In [ ]:
imagegen = ImageDataGenerator()
train = imagegen.flow_from_directory("data/train/", class_mode="categorical", shuffle=True, batch_size=64, target_size=(256, 256))
val = imagegen.flow_from_directory("data/validation/", class_mode="categorical", shuffle=True, batch_size=64, target_size=(256, 256))


In [3]:
pretrained_model = VGG16(include_top=False, weights='imagenet')
pretrained_model.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, None, None, 3)     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, None, None, 64)    1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, None, None, 64)    36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, None, None, 64)    0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, None, None, 128)   73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, None, None, 128)   147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, None, None, 128)   0     

In [4]:
vgg_features_train = pretrained_model.predict(train)
vgg_features_val = pretrained_model.predict(val)

In [5]:
train_target = to_categorical(train.labels)
val_target = to_categorical(val.labels)

In [12]:
model2 = Sequential()

model2.add(Conv2D(25, (5, 5), activation='relu', strides=(1, 1), padding='same', input_shape=(8,8,512)))
model2.add(MaxPool2D(pool_size=(2, 2), padding='same'))
model2.add(Conv2D(50, (5, 5), activation='relu', strides=(2, 2), padding='same'))
model2.add(MaxPool2D(pool_size=(2, 2), padding='same'))

model2.add(Flatten())
model2.add(Dense(100, activation='relu'))
model2.add(Dropout(0.5))
model2.add(BatchNormalization())
model2.add(Dense(3, activation='softmax'))

model2.compile(optimizer='adam', metrics=['accuracy'], loss='categorical_crossentropy')

model2.summary()

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_7 (Conv2D)            (None, 8, 8, 25)          320025    
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 4, 4, 25)          0         
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 2, 2, 50)          31300     
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 1, 1, 50)          0         
_________________________________________________________________
flatten_6 (Flatten)          (None, 50)                0         
_________________________________________________________________
dense_5 (Dense)              (None, 100)               5100      
_________________________________________________________________
dropout_3 (Dropout)          (None, 100)              

In [13]:
model2.fit(vgg_features_train, train_target, epochs=50, batch_size=128, validation_data=(vgg_features_val, val_target))


Train on 820 samples, validate on 423 samples
Epoch 1/50
820/820 [==============================] - 4s 4ms/step - loss: 1.5127 - accuracy: 0.3061 - val_loss: 5.2792 - val_accuracy: 0.3806
Epoch 2/50
820/820 [==============================] - 2s 2ms/step - loss: 1.1799 - accuracy: 0.4524 - val_loss: 3.5184 - val_accuracy: 0.3712
Epoch 3/50
820/820 [==============================] - 2s 2ms/step - loss: 1.1738 - accuracy: 0.4256 - val_loss: 2.1769 - val_accuracy: 0.3688
Epoch 4/50
820/820 [==============================] - 2s 2ms/step - loss: 1.1232 - accuracy: 0.4610 - val_loss: 1.4598 - val_accuracy: 0.3121
Epoch 5/50
820/820 [==============================] - 2s 2ms/step - loss: 1.0413 - accuracy: 0.4988 - val_loss: 1.3887 - val_accuracy: 0.3121
Epoch 6/50
820/820 [==============================] - 2s 2ms/step - loss: 0.9805 - accuracy: 0.5183 - val_loss: 1.3077 - val_accuracy: 0.3357
Epoch 7/50
820/820 [==============================] - 2s 2ms/step - loss: 0.8441 - accuracy: 0.6061 - 

## Model 3

In [23]:
imagegen = ImageDataGenerator()
train = imagegen.flow_from_directory("data/train/", class_mode="categorical", shuffle=True, batch_size=32, target_size=(256, 256))
val = imagegen.flow_from_directory("data/validation/", class_mode="categorical", shuffle=True, batch_size=32, target_size=(256, 256))


Found 730 images belonging to 3 classes.
Found 365 images belonging to 3 classes.


In [24]:
model3 = Sequential()
model3.add(InputLayer(input_shape=(256, 256, 3)))

model3.add(Conv2D(25, (5, 5), activation='relu', strides=(1, 1), padding='same')) # add batch size and ES
model3.add(MaxPool2D(pool_size=(2, 2), padding='same'))
model3.add(Conv2D(50, (5, 5), activation='relu', strides=(2, 2), padding='same'))
model3.add(MaxPool2D(pool_size=(2, 2), padding='same'))
model3.add(BatchNormalization())
model3.add(Conv2D(70, (3, 3), activation='relu', strides=(2, 2), padding='same'))
model3.add(MaxPool2D(pool_size=(2, 2), padding='valid'))
model3.add(BatchNormalization())
model3.add(Flatten())
model3.add(Dense(units=100, activation='relu'))
model3.add(Dropout(0.25))
model3.add(Dense(units=100, activation='relu'))
model3.add(Dropout(0.25))
model3.add(Dense(units=3, activation='softmax'))

model3.compile(loss='categorical_crossentropy', optimizer="adam", metrics=['accuracy'])

early_stop = EarlyStopping(monitor = "val_accuracy", 
                           patience = 10,
                           min_delta = 0,
                           restore_best_weights = True)

model3.fit_generator(train, epochs=30, validation_data=val, callbacks = [early_stop])

Epoch 1/30
23/23 [==============================] - 112s 5s/step - loss: 1.2770 - accuracy: 0.4411 - val_loss: 8.9282 - val_accuracy: 0.3945
Epoch 2/30
23/23 [==============================] - 109s 5s/step - loss: 0.9570 - accuracy: 0.5726 - val_loss: 2.8163 - val_accuracy: 0.3781
Epoch 3/30
23/23 [==============================] - 105s 5s/step - loss: 0.7846 - accuracy: 0.6534 - val_loss: 1.9285 - val_accuracy: 0.4986
Epoch 4/30
23/23 [==============================] - 103s 4s/step - loss: 0.5863 - accuracy: 0.7658 - val_loss: 1.6091 - val_accuracy: 0.5918
Epoch 5/30
23/23 [==============================] - 103s 4s/step - loss: 0.4922 - accuracy: 0.7986 - val_loss: 1.2271 - val_accuracy: 0.5479
Epoch 6/30
23/23 [==============================] - 104s 5s/step - loss: 0.3784 - accuracy: 0.8507 - val_loss: 0.9209 - val_accuracy: 0.5479
Epoch 7/30
23/23 [==============================] - 103s 4s/step - loss: 0.3023 - accuracy: 0.8863 - val_loss: 1.5286 - val_accuracy: 0.5260
Epoch 8/30
23

**We kept Model 3 because it gave us the best results.**

## Running Model 3 on Test Data

In [25]:
test = imagegen.flow_from_directory("data/test/", class_mode=None, shuffle=False, batch_size=1, target_size=(256, 256))

test.reset()
pred = model3.predict_generator(test, verbose=1)

Found 8 images belonging to 1 classes.
8/8 [==============================] - 1s 178ms/step


In [26]:
predicted_class_indices=np.argmax(pred,axis=1)


In [36]:
train.class_indices

{'Damaged': 0, 'Destroyed': 1, 'Good': 2}

In [27]:
labels = (train.class_indices)
labels = dict((v,k) for k,v in labels.items())
predictions = [labels[k] for k in predicted_class_indices]

In [35]:
filenames = test.filenames
results = pd.DataFrame({"Filename": filenames,
                        "Predictions": predictions})
results.to_csv("test_results.csv", index = False)

## Save Model 3 and Train Indices

Reference for saving train indices:<br>
https://stackoverflow.com/questions/19201290/how-to-save-a-dictionary-to-a-file/32216025

In [42]:
def save_obj(obj, name):
    with open('model/'+ name + '.pkl', 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)

save_obj(train.class_indices, "train_class_indices")

In [33]:
model3.save("./saved_model/final_model.h5")